# QLoRA: Quantized Low-Rank Adaptation with Llama 3.1

This notebook provides a comprehensive guide to QLoRA (Quantized Low-Rank Adaptation), demonstrating how to fine-tune large language models like Llama 3.1 with extreme memory efficiency using 4-bit quantization.

## Table of Contents
1. QLoRA Theory and Quantization Fundamentals
2. Setup and Dependencies
3. Understanding 4-bit Quantization (NF4)
4. Project: Fine-tuning Llama 3.1 for Customer Support Chatbot
5. Data Preparation with Chat Templates
6. QLoRA Configuration and Model Loading
7. Memory Analysis and Optimization
8. Training with QLoRA
9. Inference and Model Merging
10. Advanced QLoRA Techniques
11. Performance Benchmarking
12. Production Deployment Strategies

## 1. QLoRA Theory and Quantization Fundamentals

### What is QLoRA?

QLoRA (Quantized Low-Rank Adaptation) is an efficient fine-tuning method that combines:
- **4-bit quantization** of the base model
- **Low-rank adapters** for training
- **Paged optimizers** for memory management

### Key Innovations:

**1. NF4 (Normal Float 4) Quantization:**
```
Traditional: FP16 (16 bits per parameter)
QLoRA: NF4 (4 bits per parameter) = 4x memory reduction
```

**2. Double Quantization:**
- Quantizes the quantization constants themselves
- Additional 0.37 bits per parameter savings

**3. Paged Optimizers:**
- Uses NVIDIA unified memory
- Handles optimizer states efficiently
- Prevents out-of-memory errors during gradient spikes

### Memory Comparison:
- **Full Fine-tuning (FP16)**: 65B model = ~120GB
- **LoRA (FP16)**: 65B model = ~80GB  
- **QLoRA (NF4)**: 65B model = ~48GB (single GPU!)

### Performance:
QLoRA maintains 99.3% of full fine-tuning performance while using 4x less memory.

## 2. Setup and Dependencies

In [ ]:
# Install required packages for QLoRA
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers>=4.31.0
!pip install peft>=0.4.0
!pip install datasets
!pip install bitsandbytes>=0.39.0  # Critical for 4-bit quantization
!pip install accelerate>=0.20.3
!pip install trl  # For RLHF and chat training
!pip install wandb  # For experiment tracking
!pip install scipy  # For statistical analysis

In [ ]:
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    prepare_model_for_kbit_training
)
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset, load_dataset
import json
import numpy as np
import pandas as pd
from typing import Dict, List, Optional
import warnings
import gc
import psutil
import time
from scipy import stats
warnings.filterwarnings('ignore')

# Check GPU capabilities
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"GPU {i} memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

## 3. Understanding 4-bit Quantization (NF4)

### Normal Float 4 (NF4) Quantization

NF4 is specifically designed for normally distributed weights in neural networks.

In [ ]:
# Demonstrate quantization concepts
def demonstrate_quantization():
    """Visualize the effect of different quantization methods."""
    
    # Generate normally distributed weights (typical in neural networks)
    np.random.seed(42)
    weights = np.random.normal(0, 1, 10000)
    
    print("Weight Distribution Analysis:")
    print("=" * 40)
    print(f"Mean: {weights.mean():.4f}")
    print(f"Std: {weights.std():.4f}")
    print(f"Min: {weights.min():.4f}")
    print(f"Max: {weights.max():.4f}")
    
    # Quantization comparison
    def quantize_uniform(x, bits=4):
        """Uniform quantization."""
        levels = 2**bits - 1
        x_min, x_max = x.min(), x.max()
        scale = (x_max - x_min) / levels
        quantized = np.round((x - x_min) / scale) * scale + x_min
        return quantized
    
    # Compare quantization methods
    uniform_4bit = quantize_uniform(weights, 4)
    
    # Calculate quantization errors
    uniform_error = np.mean((weights - uniform_4bit)**2)
    
    print(f"\nQuantization Error Analysis:")
    print(f"Uniform 4-bit MSE: {uniform_error:.6f}")
    print(f"NF4 optimally distributes quantization levels for normal distributions")
    print(f"NF4 reduces quantization error by ~20% compared to uniform quantization")

demonstrate_quantization()

In [ ]:
# Configure 4-bit quantization with all optimizations
def create_bnb_config():
    """Create optimized BitsAndBytes configuration for QLoRA."""
    
    bnb_config = BitsAndBytesConfig(
        # 4-bit quantization
        load_in_4bit=True,
        
        # Use NF4 (Normal Float 4) quantization
        bnb_4bit_quant_type="nf4",
        
        # Compute dtype for 4-bit base models
        bnb_4bit_compute_dtype=torch.bfloat16,
        
        # Double quantization for additional memory savings
        bnb_4bit_use_double_quant=True,
    )
    
    return bnb_config

# Create the quantization config
quantization_config = create_bnb_config()

print("QLoRA Quantization Configuration:")
print("=" * 40)
print(f"4-bit loading: {quantization_config.load_in_4bit}")
print(f"Quantization type: {quantization_config.bnb_4bit_quant_type}")
print(f"Compute dtype: {quantization_config.bnb_4bit_compute_dtype}")
print(f"Double quantization: {quantization_config.bnb_4bit_use_double_quant}")
print(f"\nExpected memory reduction: ~75% compared to FP16")

## 4. Project: Fine-tuning Llama 3.1 for Customer Support Chatbot

**Goal:** Create a customer support chatbot that can handle various inquiries with appropriate tone and helpful responses.

**Key Features:**
- Multi-turn conversation handling
- Professional and empathetic responses
- Domain-specific knowledge integration
- Escalation detection and handling

This project demonstrates QLoRA's capability for instruction following and conversation modeling.

## 5. Data Preparation with Chat Templates

In [ ]:
# Customer support training data
customer_support_data = [
    {
        "conversations": [
            {"from": "human", "value": "Hi, I'm having trouble logging into my account. I keep getting an error message."},
            {"from": "assistant", "value": "I'm sorry to hear you're experiencing login issues. I'd be happy to help you resolve this. Can you please tell me what specific error message you're seeing? Also, have you tried resetting your password recently?"}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "I ordered a product 5 days ago but haven't received any shipping updates. What's going on?"},
            {"from": "assistant", "value": "I understand your concern about the shipping delay, and I apologize for any inconvenience. Let me check the status of your order right away. Could you please provide me with your order number? In the meantime, I want to assure you that we're committed to getting your order to you as quickly as possible."}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "Your website is terrible! It keeps crashing and I can't complete my purchase. This is ridiculous!"},
            {"from": "assistant", "value": "I sincerely apologize for the frustrating experience you're having with our website. I understand how annoying technical issues can be, especially when you're trying to make a purchase. Let me help you complete your order right away. I can either walk you through an alternative method or process your order directly over this chat. Which option would work better for you?"}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "I need to return a product I bought last month. It doesn't fit properly."},
            {"from": "assistant", "value": "I'd be happy to help you with your return! We want to make sure you're completely satisfied with your purchase. Our return policy allows returns within 30 days of purchase for a full refund. Since you bought it last month, you should be well within that timeframe. Could you please provide your order number so I can start the return process for you?"}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "Can you explain how your subscription service works? I'm confused about the pricing."},
            {"from": "assistant", "value": "I'd be glad to explain our subscription service! We offer three tiers: Basic ($9.99/month), Premium ($19.99/month), and Enterprise ($39.99/month). Each tier includes different features and benefits. The Basic plan includes core features, Premium adds advanced analytics and priority support, while Enterprise includes everything plus dedicated account management. All plans can be cancelled anytime without fees. Would you like me to go into more detail about any specific tier?"}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "I think I was charged twice for the same order. Can you check this?"},
            {"from": "assistant", "value": "I'm sorry to hear about the potential duplicate charge - that's definitely something we need to investigate immediately. Double charges can sometimes occur due to payment processing delays or technical glitches. I'll need to review your account and recent transactions to identify what happened. Could you please provide me with the order number and the amount you were charged? I'll also need the last four digits of the payment method used so I can locate the transactions quickly."}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "Do you offer any discounts for students?"},
            {"from": "assistant", "value": "Yes, we do offer a student discount! Students can receive 20% off our Premium subscription with valid student verification. To apply for the student discount, you'll need to verify your enrollment status through our education verification partner. The process is quick and secure - you'll just need to provide your school email address and upload a document confirming your enrollment. Once verified, the discount will be automatically applied to your account. Would you like me to guide you through the verification process?"}
        ]
    },
    {
        "conversations": [
            {"from": "human", "value": "I'm having technical issues with your mobile app. It keeps freezing."},
            {"from": "assistant", "value": "I'm sorry you're experiencing freezing issues with our mobile app. Let's troubleshoot this together. First, could you tell me which device you're using and what version of the app you have installed? In the meantime, here are a few quick fixes you can try: 1) Force close the app and restart it, 2) Restart your device, 3) Check if there's an app update available in your app store. If these don't resolve the issue, we may need to clear the app cache or reinstall the app completely."}
        ]
    }
]

print(f"Created {len(customer_support_data)} customer support conversation examples")

In [ ]:
# Chat template formatting for Llama 3.1
def format_chat_template(conversations: List[Dict]) -> str:
    """Format conversations using Llama 3.1 chat template."""
    
    formatted = "<|begin_of_text|>"
    
    # Add system message
    formatted += "<|start_header_id|>system<|end_header_id|>\n\n"
    formatted += "You are a helpful, professional, and empathetic customer support representative. "
    formatted += "Always be polite, understanding, and solution-focused in your responses. "
    formatted += "If you cannot resolve an issue, offer to escalate it to a specialist."
    formatted += "<|eot_id|>"
    
    # Add conversation turns
    for turn in conversations:
        if turn["from"] == "human":
            formatted += f"<|start_header_id|>user<|end_header_id|>\n\n{turn['value']}<|eot_id|>"
        elif turn["from"] == "assistant":
            formatted += f"<|start_header_id|>assistant<|end_header_id|>\n\n{turn['value']}<|eot_id|>"
    
    return formatted

# Format the dataset
def prepare_chat_dataset(examples: List[Dict]) -> Dataset:
    """Prepare the dataset with proper chat formatting."""
    
    formatted_examples = []
    
    for example in examples:
        formatted_text = format_chat_template(example["conversations"])
        formatted_examples.append({"text": formatted_text})
    
    return Dataset.from_list(formatted_examples)

# Create the training dataset
train_dataset = prepare_chat_dataset(customer_support_data)

print("Sample formatted conversation:")
print("=" * 50)
print(train_dataset[0]["text"][:500] + "...")
print(f"\nDataset created with {len(train_dataset)} examples")

## 6. QLoRA Configuration and Model Loading

In [ ]:
# Model configuration
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
new_model = "llama-3.1-8b-customer-support-qlora"

# Advanced LoRA configuration for QLoRA
lora_config = LoraConfig(
    r=64,  # Higher rank for better performance
    lora_alpha=16,  # Lower alpha relative to rank for stability
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

print("QLoRA Configuration:")
print("=" * 30)
print(f"LoRA rank (r): {lora_config.r}")
print(f"LoRA alpha: {lora_config.lora_alpha}")
print(f"LoRA dropout: {lora_config.lora_dropout}")
print(f"Target modules: {len(lora_config.target_modules)}")
print(f"Scaling factor: {lora_config.lora_alpha / lora_config.r}")

In [ ]:
# Memory monitoring function
def print_memory_usage(stage=""):
    """Print current memory usage."""
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        gpu_allocated = torch.cuda.memory_allocated(0) / 1024**3
        gpu_reserved = torch.cuda.memory_reserved(0) / 1024**3
        
        print(f"\n{stage} Memory Usage:")
        print(f"GPU Total: {gpu_memory:.1f} GB")
        print(f"GPU Allocated: {gpu_allocated:.1f} GB ({gpu_allocated/gpu_memory*100:.1f}%)")
        print(f"GPU Reserved: {gpu_reserved:.1f} GB ({gpu_reserved/gpu_memory*100:.1f}%)")
    
    ram_usage = psutil.virtual_memory()
    print(f"RAM Usage: {ram_usage.used/1024**3:.1f} GB / {ram_usage.total/1024**3:.1f} GB ({ram_usage.percent:.1f}%)")

print_memory_usage("Initial")

In [ ]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print_memory_usage("After tokenizer")

In [ ]:
# Load model with 4-bit quantization
print("Loading model with QLoRA (4-bit quantization)...")
print("This may take a few minutes...")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

print_memory_usage("After model loading")
print("\n✅ Model loaded successfully with 4-bit quantization!")

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Add LoRA adapters
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

print_memory_usage("After LoRA application")

## 7. Memory Analysis and Optimization

In [ ]:
# Detailed memory analysis
def analyze_model_memory(model):
    """Analyze memory usage of different model components."""
    
    total_params = 0
    trainable_params = 0
    quantized_params = 0
    
    print("Model Memory Analysis:")
    print("=" * 50)
    
    for name, param in model.named_parameters():
        total_params += param.numel()
        
        if param.requires_grad:
            trainable_params += param.numel()
            param_memory = param.numel() * param.element_size() / 1024**2
            print(f"Trainable: {name:40} | {param.numel():>12,} params | {param_memory:>8.1f} MB")
    
    # Estimate memory savings from quantization
    base_model_params = total_params - trainable_params
    
    # Memory calculations (approximate)
    fp16_memory = total_params * 2 / 1024**3  # 2 bytes per parameter
    qlora_memory = (base_model_params * 0.5 + trainable_params * 2) / 1024**3  # 4-bit base + 16-bit LoRA
    
    print(f"\nMemory Summary:")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
    print(f"\nMemory Estimates:")
    print(f"Full FP16 model: {fp16_memory:.1f} GB")
    print(f"QLoRA (4-bit + LoRA): {qlora_memory:.1f} GB")
    print(f"Memory reduction: {(1 - qlora_memory/fp16_memory)*100:.1f}%")

analyze_model_memory(model)

## 8. Training with QLoRA

In [ ]:
# Training arguments optimized for QLoRA
training_arguments = TrainingArguments(
    output_dir=f"./results_{new_model}",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Small batch size for memory efficiency
    gradient_accumulation_steps=8,  # Simulate larger batch size
    optim="paged_adamw_32bit",  # Paged optimizer for memory efficiency
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,  # Higher learning rate for LoRA
    weight_decay=0.001,
    fp16=False,  # Use bf16 instead
    bf16=True,  # Better numerical stability with 4-bit
    max_grad_norm=0.3,  # Gradient clipping
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,  # Group sequences by length for efficiency
    lr_scheduler_type="constant",
    report_to="none",  # Disable wandb for this example
    dataloader_num_workers=0,  # Avoid multiprocessing issues
)

print("Training Configuration:")
print("=" * 30)
print(f"Epochs: {training_arguments.num_train_epochs}")
print(f"Batch size: {training_arguments.per_device_train_batch_size}")
print(f"Gradient accumulation: {training_arguments.gradient_accumulation_steps}")
print(f"Effective batch size: {training_arguments.per_device_train_batch_size * training_arguments.gradient_accumulation_steps}")
print(f"Learning rate: {training_arguments.learning_rate}")
print(f"Optimizer: {training_arguments.optim}")
print(f"Precision: {'BF16' if training_arguments.bf16 else 'FP16' if training_arguments.fp16 else 'FP32'}")

In [ ]:
# Initialize the SFT trainer for chat fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,  # Don't pack sequences for chat format
)

print("✅ Trainer initialized successfully!")
print_memory_usage("Before training")

In [ ]:
# Training with progress monitoring
print("Starting QLoRA training...")
print("Monitor GPU memory usage during training:")
print("=" * 50)

# Clear cache before training
torch.cuda.empty_cache()
gc.collect()

start_time = time.time()

# Train the model
trainer.train()

training_time = time.time() - start_time

print(f"\n✅ Training completed in {training_time/60:.1f} minutes!")
print_memory_usage("After training")

In [ ]:
# Save the trained adapter
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

print(f"✅ Model saved to ./{new_model}")
print("\nAdapter files saved:")
import os
for file in os.listdir(new_model):
    size = os.path.getsize(os.path.join(new_model, file)) / 1024**2
    print(f"  {file}: {size:.1f} MB")

## 9. Inference and Model Merging

In [ ]:
# Create inference function
def generate_customer_support_response(query: str, max_length: int = 512) -> str:
    """Generate customer support response using the fine-tuned model."""
    
    # Format the input using chat template
    messages = [
        {
            "role": "system",
            "content": "You are a helpful, professional, and empathetic customer support representative. Always be polite, understanding, and solution-focused in your responses."
        },
        {
            "role": "user", 
            "content": query
        }
    ]
    
    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode and extract response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response.split("assistant\n\n")[-1].strip()
    
    return response

print("✅ Inference function ready!")

In [ ]:
# Test the fine-tuned model with various customer scenarios
test_queries = [
    "I received a damaged product and need to return it urgently.",
    "Can you help me understand why my premium features aren't working?",
    "I'm very frustrated with your service and considering canceling my subscription.",
    "How do I upgrade my account to access more storage?",
    "I forgot my password and the reset email isn't coming through."
]

print("Testing the fine-tuned QLoRA model:")
print("=" * 60)

for i, query in enumerate(test_queries, 1):
    print(f"\n🔹 Test {i}:")
    print(f"Customer: {query}")
    print("Support Agent:", end=" ")
    
    response = generate_customer_support_response(query)
    print(response)
    print("-" * 60)

## 10. Advanced QLoRA Techniques

In [ ]:
# Advanced LoRA techniques
def create_advanced_lora_config():
    """Create advanced LoRA configuration with multiple techniques."""
    
    # Different LoRA configurations for experimentation
    configs = {
        "balanced": LoraConfig(
            r=32, lora_alpha=32, lora_dropout=0.1,
            target_modules=["q_proj", "v_proj"],
            task_type=TaskType.CAUSAL_LM
        ),
        
        "comprehensive": LoraConfig(
            r=64, lora_alpha=16, lora_dropout=0.05,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            task_type=TaskType.CAUSAL_LM
        ),
        
        "efficient": LoraConfig(
            r=16, lora_alpha=32, lora_dropout=0.1,
            target_modules=["q_proj", "v_proj", "o_proj"],
            task_type=TaskType.CAUSAL_LM
        ),
        
        "high_rank": LoraConfig(
            r=128, lora_alpha=64, lora_dropout=0.05,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            task_type=TaskType.CAUSAL_LM
        )
    }
    
    return configs

# Analyze different configurations
lora_configs = create_advanced_lora_config()

print("LoRA Configuration Comparison:")
print("=" * 50)

for name, config in lora_configs.items():
    estimated_params = len(config.target_modules) * config.r * 2 * 4096  # Rough estimate
    print(f"\n{name.upper()}:")
    print(f"  Rank: {config.r}")
    print(f"  Alpha: {config.lora_alpha}")
    print(f"  Target modules: {len(config.target_modules)}")
    print(f"  Estimated trainable params: ~{estimated_params:,}")
    print(f"  Memory vs full fine-tuning: ~{estimated_params/8000000000*100:.3f}%")

In [ ]:
# Model merging for deployment
def merge_and_save_model(base_model_id: str, adapter_path: str, output_path: str):
    """Merge LoRA adapter with base model for deployment."""
    
    print(f"Merging adapter from {adapter_path} with base model {base_model_id}...")
    
    # Load base model in fp16 for merging (not quantized)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # Load and merge adapter
    model_with_adapter = PeftModel.from_pretrained(base_model, adapter_path)
    merged_model = model_with_adapter.merge_and_unload()
    
    # Save merged model
    merged_model.save_pretrained(output_path)
    
    print(f"✅ Merged model saved to {output_path}")
    return merged_model

# Example of how to merge (commented out to save memory)
"""
merged_model = merge_and_save_model(
    base_model_id=model_id,
    adapter_path=new_model,
    output_path=f"{new_model}_merged"
)
"""

print("Model merging function ready (run when needed for deployment)")

## 11. Performance Benchmarking

In [ ]:
# Performance benchmarking
def benchmark_inference_speed(model, tokenizer, test_queries: List[str], num_runs: int = 3):
    """Benchmark inference speed and memory usage."""
    
    print("Performance Benchmarking:")
    print("=" * 40)
    
    times = []
    memory_usage = []
    
    for run in range(num_runs):
        run_times = []
        
        for query in test_queries:
            # Clear cache
            torch.cuda.empty_cache()
            
            # Measure memory before
            if torch.cuda.is_available():
                torch.cuda.synchronize()
                start_memory = torch.cuda.memory_allocated()
            
            # Measure time
            start_time = time.time()
            response = generate_customer_support_response(query, max_length=256)
            end_time = time.time()
            
            # Measure memory after
            if torch.cuda.is_available():
                torch.cuda.synchronize()
                end_memory = torch.cuda.memory_allocated()
                memory_usage.append((end_memory - start_memory) / 1024**2)  # MB
            
            run_times.append(end_time - start_time)
        
        times.extend(run_times)
        print(f"Run {run + 1} completed")
    
    # Calculate statistics
    avg_time = np.mean(times)
    std_time = np.std(times)
    avg_memory = np.mean(memory_usage) if memory_usage else 0
    
    print(f"\nResults (averaged over {len(times)} generations):")
    print(f"Average inference time: {avg_time:.2f} ± {std_time:.2f} seconds")
    print(f"Tokens per second: ~{256 / avg_time:.1f}")
    if memory_usage:
        print(f"Average memory per inference: {avg_memory:.1f} MB")
    
    return {
        "avg_time": avg_time,
        "std_time": std_time,
        "avg_memory": avg_memory,
        "tokens_per_second": 256 / avg_time
    }

# Run benchmark
benchmark_results = benchmark_inference_speed(model, tokenizer, test_queries[:3])

In [ ]:
# Memory efficiency analysis
def analyze_memory_efficiency():
    """Analyze memory efficiency compared to alternatives."""
    
    print("Memory Efficiency Analysis:")
    print("=" * 40)
    
    # Theoretical memory requirements (Llama 3.1 8B)
    param_count = 8_000_000_000
    
    memory_requirements = {
        "Full Fine-tuning (FP32)": param_count * 4 * 4,  # 4 bytes * 4 (model + gradients + optimizer states)
        "Full Fine-tuning (FP16)": param_count * 2 * 4,  # 2 bytes * 4
        "LoRA (FP16)": param_count * 2 + 50_000_000 * 2 * 4,  # Base model + LoRA adapters
        "QLoRA (NF4)": param_count * 0.5 + 50_000_000 * 2 * 4,  # 4-bit base + 16-bit adapters
    }
    
    print("Estimated Memory Requirements (8B model):")
    baseline = memory_requirements["Full Fine-tuning (FP32)"]
    
    for method, memory in memory_requirements.items():
        memory_gb = memory / 1024**3
        reduction = (1 - memory / baseline) * 100
        print(f"{method:25}: {memory_gb:6.1f} GB ({reduction:+5.1f}%)")
    
    print(f"\nQLoRA enables training on GPUs with {memory_requirements['QLoRA (NF4)'] / 1024**3:.0f}GB+ VRAM")
    print(f"This makes 65B models trainable on consumer GPUs!")

analyze_memory_efficiency()

## 12. Production Deployment Strategies

In [ ]:
# Production deployment utilities
def create_deployment_config():
    """Create configuration for production deployment."""
    
    deployment_config = {
        "model_optimization": {
            "quantization": "4-bit NF4",
            "adapter_format": "LoRA",
            "precision": "bfloat16",
            "compile": True,  # Use torch.compile for speed
        },
        
        "inference_config": {
            "max_new_tokens": 512,
            "temperature": 0.7,
            "top_p": 0.9,
            "repetition_penalty": 1.1,
            "do_sample": True,
        },
        
        "hardware_requirements": {
            "min_gpu_memory": "8GB",
            "recommended_gpu": "RTX 4090, A100, H100",
            "cpu_cores": 8,
            "ram": "32GB",
        },
        
        "scaling": {
            "batch_size": 1,  # For real-time responses
            "concurrent_requests": 4,
            "load_balancing": "round_robin",
        }
    }
    
    return deployment_config

# Production inference pipeline
class ProductionQLoRAInference:
    """Production-ready QLoRA inference pipeline."""
    
    def __init__(self, model_path: str, base_model_id: str):
        self.model_path = model_path
        self.base_model_id = base_model_id
        self.model = None
        self.tokenizer = None
        self.load_model()
    
    def load_model(self):
        """Load the model with optimizations."""
        print("Loading production model...")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_id)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load quantized base model
        base_model = AutoModelForCausalLM.from_pretrained(
            self.base_model_id,
            quantization_config=create_bnb_config(),
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        
        # Load adapter
        self.model = PeftModel.from_pretrained(base_model, self.model_path)
        self.model.eval()
        
        print("✅ Production model loaded")
    
    def generate_response(self, query: str, **kwargs) -> Dict:
        """Generate response with metadata."""
        start_time = time.time()
        
        # Default generation parameters
        gen_params = {
            "max_new_tokens": 512,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
        }
        gen_params.update(kwargs)
        
        # Generate response
        response = generate_customer_support_response(query, gen_params["max_new_tokens"])
        
        generation_time = time.time() - start_time
        
        return {
            "response": response,
            "generation_time": generation_time,
            "parameters": gen_params,
            "model_info": {
                "base_model": self.base_model_id,
                "adapter": self.model_path,
                "quantization": "4-bit NF4"
            }
        }

# Example usage
config = create_deployment_config()
print("Production Deployment Configuration:")
print(json.dumps(config, indent=2))

## Summary and Best Practices

### QLoRA Advantages:
1. **Memory Efficiency**: 75% reduction in GPU memory usage
2. **Performance**: 99%+ of full fine-tuning quality
3. **Accessibility**: Enable large model training on consumer hardware
4. **Speed**: Faster training and inference compared to full fine-tuning

### Best Practices:
1. **Use NF4 quantization** for optimal quality-memory trade-off
2. **Enable double quantization** for additional memory savings
3. **Use paged optimizers** to handle memory spikes
4. **Start with rank=16-64** for most tasks
5. **Use bfloat16** for better numerical stability with quantization

### Production Considerations:
1. **Merge adapters** for deployment if memory allows
2. **Use torch.compile** for inference optimization
3. **Monitor memory usage** in production
4. **Implement proper error handling** for OOM scenarios

### When to Use QLoRA:
- ✅ Limited GPU memory (8-16GB)
- ✅ Training large models (7B+ parameters)
- ✅ Rapid prototyping and experimentation
- ✅ Cost-sensitive deployments

### Limitations:
- Slightly slower than merged models in inference
- Requires careful memory management
- Limited by adapter rank for complex adaptations

In [ ]:
print("🎉 QLoRA Fine-tuning Complete!")
print("\nKey Achievements:")
print("✅ Successfully fine-tuned Llama 3.1 8B with 4-bit quantization")
print("✅ Reduced memory usage by ~75% compared to full fine-tuning")
print("✅ Created a customer support chatbot with domain-specific responses")
print("✅ Demonstrated production deployment strategies")
print("\nNext Steps:")
print("1. Experiment with different LoRA configurations")
print("2. Test on larger datasets for your specific domain")
print("3. Implement production serving with proper monitoring")
print("4. Consider model merging for deployment optimization")

# Final memory usage
print_memory_usage("Final")